In [ ]:
%pip -q install google-genai

In [ ]:
# --- Part 1: Installation and Setup ---
!pip install -q google-adk google-genai beautifulsoup4 requests

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# --- Part 2: Import Libraries and Helper Functions ---
from google.adk.agents import Agent, Tool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types, GenerativeModel
import textwrap
from IPython.display import display, Markdown
import requests
from bs4 import BeautifulSoup
import json
import time

# --- Helper Functions ---
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# --- Part 3: Data Scraping Tools ---
def scrape_metacritic(game_title):
    try:
        search_url = f"https://www.metacritic.com/search/game/{game_title}/results"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, "html.parser")
        first_result = soup.find("a", class_="result_title")
        if first_result:
            game_url = "https://www.metacritic.com" + first_result["href"]
            game_response = requests.get(game_url)
            game_soup = BeautifulSoup(game_response.content, "html.parser")
            critic_score = game_soup.find("span", itemprop="ratingValue").text.strip()
            user_score_element = game_soup.find("div", class_="userscore_wrap feature_userscore")
            user_score = user_score_element.find("div", class_="metascore_w user large game").text.strip() if user_score_element else "N/A"
            return {"critic_score": critic_score, "user_score": user_score}
        else:
            return None
    except Exception as e:
        print(f"Error scraping Metacritic for {game_title}: {e}")
        return None

def scrape_howlongtobeat(game_title):
    try:
        search_url = f"https://howlongtobeat.com/?q={game_title.replace(' ', '+')}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, "html.parser")
        first_result = soup.find("a", href=True)
        if first_result:
            game_url = "https://howlongtobeat.com/" + first_result["href"]
            game_response = requests.get(game_url)
            game_soup = BeautifulSoup(game_response.content, "html.parser")
            times = game_soup.find_all("div", class_="game_times")
            if times:
                main_story_element = times[0].find("li", class_="time_100")
                main_story = main_story_element.text.strip() if main_story_element else "N/A"
                main_plus_extras_element = times[1].find("li", class_="time_100")
                main_plus_extras = main_plus_extras_element.text.strip() if main_plus_extras_element else "N/A"
                completionist_element = times[2].find("li", class_="time_100")
                completionist = completionist_element.text.strip() if completionist_element else "N/A"
                return {"main_story": main_story, "main_plus_extras": main_plus_extras, "completionist": completionist}
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error scraping HowLongToBeat for {game_title}: {e}")
        return None

def scrape_steam_reviews(game_title):  # Simplified Steam scraping
    # Needs a more robust approach (e.g., SteamSpy, Steam API - if accessible)
    return {"average_score": "N/A", "num_reviews": "N/A", "positive_percentage": "N/A", "summary": "N/A"}

metacritic_tool = Tool(
    name="metacritic_scraper",
    description="Tool to get Metacritic scores for a game.",
    func=scrape_metacritic,
)

hltb_tool = Tool(
    name="howlongtobeat_scraper",
    description="Tool to get HowLongToBeat completion times for a game.",
    func=scrape_howlongtobeat,
)

steam_reviews_tool = Tool(
    name="steam_reviews_scraper",
    description="Tool to get Steam reviews for a game.",
    func=scrape_steam_reviews,
)

In [ ]:
# --- Part 4: Agent Definitions ---
llm = GenerativeModel(model_name="gemini-2.5-pro-preview-03-25")  # Initialize the LLM

game_info_agent = Agent(
    name="game_info_agent",
    llm=llm,
    tools=[google_search, metacritic_tool, hltb_tool, steam_reviews_tool],
    instruction="""
    You are an expert game information retriever. You can answer questions about
    specific games, find games based on descriptions, and compare games.
    Use the available tools (google_search, metacritic_scraper, howlongtobeat_scraper,
    steam_reviews_scraper) to gather comprehensive information.
    """,
    description="Agent to retrieve detailed information about video games."
)

recommendation_agent = Agent(
    name="recommendation_agent",
    llm=llm,
    tools=[google_search],
    instruction="""
    You are a game recommendation specialist. Based on user requests and information
    gathered by other agents, provide personalized game recommendations.
    Consider user preferences, game features, critical reception, and player feedback.
    """,
    description="Agent to provide game recommendations."
)

comparison_agent = Agent(
    name="comparison_agent",
    llm=llm,
    tools=[google_search, metacritic_tool, hltb_tool],
    instruction="""
    You are a game comparison expert. Compare and contrast video games based on
    various criteria such as gameplay, story, graphics, and user reviews.
    Provide detailed and objective comparisons.
    """,
    description="Agent to compare video games."
)

🎮 Welcome to the AI Game Recommendation System! 🎮


In [ ]:
# --- Part 5: Main Flow ---
print("🎮 Welcome to the Advanced AI Game Assistant! 🎮")

session_service = InMemorySessionService()
user_id = "user1"
session_id = "game_session"

def run_interaction(user_input):
    global session_id  # Declare session_id as global

    # Determine which agent to use based on the user input
    if "compare" in user_input.lower():
        agent = comparison_agent
    elif "recommend" in user_input.lower() or "suggest" in user_input.lower() or "find" in user_input.lower():
        agent = recommendation_agent
    else:
        agent = game_info_agent

    session = session_service.create_session(app_name=agent.name, user_id=user_id, session_id=session_id)
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=user_input)])

    response = ""
    for event in runner.run(user_id=user_id, session_id=session_id, new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    response += part.text + "\n"
    return response

while True:
    user_input = input("What can I help you with today? (or type 'exit'): ")
    if user_input.lower() == "exit":
        break

    response = run_interaction(user_input)
    print("\n--- 🤖 AI Assistant Response ---\n")
    display(to_markdown(response))
    print("--------------------------------------------------------------")

print("Thank you for using the AI Game Assistant!")